In [53]:
import pandas as pd
import pickle
import os
import numpy as np
import random

In [54]:
def load_data_one_stock(path):
 
  file = open(path, 'rb')
  load_data= pickle.load(file)
  file.close()

  return load_data
  

In [79]:
# Settings 
# stock_name_list  = ['AAPL','V', 'AMD', 'MSFT', 'MA', 'NVDA']
# stock_name_list  = ['GOOGL','CVS', 'AMZN', 'CRM', 'DIS', 'MCD']

# stock_name_list  = ['AAPL','V', 'AMD', 'MSFT', 'MA', 'NVDA'] + ['GOOGL','CVS', 'AMZN', 'CRM', 'DIS', 'MCD']
stock_name_list = []
stock_name_list += ['GOOG','JPM','XOM','UNH','JNJ','V','AVGO','PG','LLY','MA','HD','CVX','MRK', 
                       'PEP','COST','ABBV','ADBE','KO','CRM','WMT','MCD','CSCO','BAC','PFE','TMO','ACN','NFLX','ABT','AMD','LIN','ORCL','CMCSA',
                       'TXN','DIS','WFC','DHR','PM','NEE','VZ','INTC','RTX','HON','LOW','UPS','INTU','SPGI','NKE','COP','QCOM','BMY','CAT','UNP','BA','ISRG',
                        'GE','IBM','AMGN','AMAT','MDT','SBUX','PLD','NOW','MS','DE','BLK','GS','T','LMT','AXP','BKNG','SYK','ADI','TJX','ELV','MDLZ','GILD','ADP','MMC',
                        'C','AMT','CVS','VRTX','SCHW','LRCX','MO','TMUS','SLB', 'ETN', 'ZTS', 'CI', 'PYPL']

stock_name_list += ['FI','CB','SO','REGN','BSX','EQIX','BDX','PANW','DUK','EOG','MU','AON','ITW','CSX','SNPS','PGR','APD','KLAC','CME','NOC','CDNS','ICE',
                       'CL','SHW','WM','HCA','TGT','FCX','FDX','F','ORLY','MMM','CMG','EW','GM','MCK','NXPI','MCO','NSC','HUM','EMR','DXCM','PNC','PH','MPC','APH',
                       'ROP','FTNT','MCHP','PXD','USB','CCI','MAR','MSI','GD','PSA','JCI','PSX','SRE','ADSK','AZO','TDG','ECL','AJG','KMB','TEL','TT','AEP','EL','PCAR',
                       'OXY','TFC','CARR','D','IDXX','GIS','ON','COF','ADM','MNST','NUE','CTAS','AIG','EXC','VLO','MRNA','ANET','WMB','O','STZ','IQV','HLT','CHTR','WELL',
                       'BIIB','SPG','MSCI','DHI','ROK']

folder_saved_models = f'historical_data\\period2yinterval1h'
parent_path = os.getcwd()
buy_money = 1000
commission_value = 0.0025
trainig_part = 0.5
test_part = 0.5

Exlude companies with bad ration and retraining!!!

Currten best settings:
{'is_near_global_max_prt': 0, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.2, 'buy_ratio_border': 10}
-3458.0764813327787 495


In [56]:
## Load data

# file_name = 'df_stock_AAPL_period2y_interval1h.pkl'
# path = os.path.join(parent_path, folder_saved_models, file_name)
## stock_name = item[str.find(item, 'df_') + 3 : str.find(item, '_20')]
# df = load_data_one_stock(path)
## df.plot()
## print(df)

In [57]:
def is_near_global_max(i, k=400, prt=70):

  if i > k:
   gmax = max(df['close'].iloc[i - k: i])
   reference_point = df['close'].iloc[i - k]
  else:
   reference_point = df['close'].iloc[0]
   gmax = max(df['close'].iloc[0: i])
 
  if gmax == reference_point:
    result = False
  elif 100 * (df['close'].iloc[i] - reference_point) / (gmax - reference_point) > prt:
    result = True
  else:
    result = False

  return result

In [81]:
def one_stock_simulation(df, ticker, mode, print_sim_results=False, print_trans_history=False):
  '''
    Parameters:
      mode: train / full / test
  '''

  history_log = {'number success': 0,
                'number fails': 0,
                'profit': 0,
                'transactions': [] }  
  last_top = df['close'].iloc[0]
  last_top_time = df['close'].index[0]
  last_top_i = 0
  reverse_flag = False
  sell_finished = True
  
  if mode == 'train':
    range_ = range(2, int(df.shape[0] * trainig_part))
  if mode == 'full': 
    range_ = range(2, int(df.shape[0] * trainig_part))
  if mode == 'test':
    range_ = range(int(df.shape[0] * trainig_part) + 1, df.shape[0])

  for i in range_:

    # dynaminc profit/loose coefficient
    if i > 200:
      if df['close'].iloc[i] / df['close'].iloc[i - 200] > bull_trend_coef: # bull trend
        gain_coef = 1.02
        lose_coef = 0.98  # not equal to 1 - k (DO Different Expertiments)
      else: # bear trend
        gain_coef = 1.005
        lose_coef = 0.95
    else: # undefined trend
      gain_coef = 1.02
      lose_coef = 0.98

    if sell_finished == False and df['high'].iloc[i] / buy_price > gain_coef:
      sell_price = float(df['high'].iloc[i])
      history_log['number success'] += 1
      number_buy_stocks = int(buy_money / buy_price)
      history_log['profit'] += (sell_price - buy_price) * number_buy_stocks - commission_value * (sell_price + buy_price) * number_buy_stocks
      history_log['transactions'].append({
        'type': 'success',
        'buy_price' : round(buy_price, 2),
        'sell_price': round(sell_price, 2), 
        'ratio': round(buy_ratio_hl ,2),
        'buy_time' : buy_time,
        'sell_time' : df['high'].index[i]
     })
      sell_finished = True

    if sell_finished == False and df['low'].iloc[i] / buy_price < lose_coef :
      sell_price = float(df['low'].iloc[i])
      sell_finished = True
      number_buy_stocks = int(buy_money / buy_price)
      history_log['profit'] += (sell_price - buy_price) * number_buy_stocks - commission_value * (sell_price + buy_price) * number_buy_stocks
      history_log['number fails'] += 1
      history_log['transactions'].append({
        'type': 'fail',
        'buy_price' : round(buy_price, 2),
        'sell_price': round(sell_price, 2),
        'ratio': round(buy_ratio_hl ,2),
        'buy_time' : buy_time,
        'sell_time' : df['low'].index[i]
      })

    # KEY parameters: RIV, prt, last_top_ratio
    # buy_time_cond = (df.index[i]).hour <= 15 and (df.index[i]).hour >= 11

    buy_ratio = float(last_top -  df['open'].iloc[i]) / float(df['ha_pct'].iloc[i])
    if df['ha_pct'].iloc[i] > RIV \
      and reverse_flag==False \
      and sell_finished == True \
      and last_top / df['open'].iloc[i] > last_top_ratio \
      and i - last_top_i > distance_from_last_top \
      and buy_time_cond \
      and buy_ratio > buy_ratio_border \
      and not(is_near_global_max(i, k=400, prt=is_near_global_max_prt)):

      reverse_flag = True
      buy_price = float(df['close'].iloc[i])
      buy_time = df.index[i]
      buy_ratio_hl = buy_ratio
      sell_finished = False

    # last top and reverse flag
    if df['ha_colour'].iloc[i] == 'red' \
      and df['ha_colour'].iloc[i - 1] == 'green'\
      and df['ha_colour'].iloc[i - 2] == 'green'\
      and df['ha_pct'].iloc[i - 1] > 0.1 \
      and df['ha_pct'].iloc[i - 2] > 0.1:
  
      last_top = df['high'].iloc[i - 1]
      last_top_i = i - 1
      reverse_flag = False
  
  # Simulation results 

  if print_sim_results:
    print('-'*50)
    print(f'Simulation results for ticker {ticker}:')
    print(history_log)
  
  if print_trans_history:
    for tran in history_log['transactions']:
      print(tran)

  return history_log['profit'], history_log['number fails']


In [107]:
buy_time_cond = True
buy_ratio_border = 7
distance_from_last_top  = 0  # NOT SIGNIFICANT?
last_top_ratio = 1.002 # last top / current open price  NOT SIGNIFICANT?
RIV = 0.1 # Reversal indication Value
is_near_global_max_prt = 96 # usd to be 96
bull_trend_coef = 1.12

best_params = {'is_near_global_max_prt': 0, 'distance_from_last_top': 2, 'last_top_ratio': 1, 'RIV': 0.2, 'buy_ratio_border': 10, 'bull_trend_coef': 1.12}
for param in best_params:
  vars()[param] = best_params[param]

parameters_list_search = {
  'is_near_global_max_prt' : [0, 20, 30, 50, 70, 80, 96, 100],
  'distance_from_last_top': [0,2,3,5],
  'last_top_ratio': [1, 1.001, 1.002, 1.003],
  'RIV' : [0.05, 0.1, 0.15, 0.2, 0.25],
  'buy_ratio_border' : [0,3,5,7,9,10],
  'bull_trend_coef': [1.12]
}

parameters_list_search = {
  'bull_trend_coef': np.arange(1.08, 1.25, 0.01)
}

In [105]:
# stock_name_list  = ['V']
best_profit = -10**5
best_profit_improved = True
optimize_profit = True
# for parameter in parameters_list_search:
while best_profit_improved:
  for i in random.sample(range(len(parameters_list_search)), len(parameters_list_search)):
    parameter = list(parameters_list_search.keys())[i]
    total_profit_list = []
    total_loses_list = []
    for value in parameters_list_search[parameter]:
      vars()[parameter] = value
      print(f'parametr is {parameter} value is {value}:')
      total_profit = 0
      total_loses = 0
      for ticker in stock_name_list:
        file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
        path = os.path.join(parent_path, folder_saved_models, file_name)
        df = load_data_one_stock(path)
        profit, loses= one_stock_simulation(df, ticker, mode='train')
        total_profit += profit
        total_loses += loses
      total_profit_list.append(total_profit)
      total_loses_list.append(total_loses)
      print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}')
    print(total_profit_list)
    if optimize_profit:
      vars()[parameter] = parameters_list_search[parameter][np.argmax(total_profit_list)]
    else:
      vars()[parameter] = parameters_list_search[parameter][np.argmin(total_loses_list)]
    print(f'parameter {parameter} best value is {vars()[parameter]}')
    # np.max(total_profit_list), np.argmax(total_profit_list)

  parameters_list_best = {}
  for parameter in parameters_list_search:
    print(f'parameter {parameter} best value is {vars()[parameter]}')
    
    parameters_list_best[parameter] = vars()[parameter]
    
  print(parameters_list_best)
  print(max(total_profit_list), min(total_loses_list))

  best_profit_improved = max(total_profit_list) > best_profit
  best_profit = max(total_profit_list) if best_profit_improved else best_profit


parametr is bull_trend_coef value is 1.08:
Total Profit is 1273.18, total loses is 465
parametr is bull_trend_coef value is 1.09:
Total Profit is 1426.52, total loses is 457
parametr is bull_trend_coef value is 1.1:
Total Profit is 1570.71, total loses is 448
parametr is bull_trend_coef value is 1.11:
Total Profit is 1604.14, total loses is 442
parametr is bull_trend_coef value is 1.12:
Total Profit is 1668.96, total loses is 438
parametr is bull_trend_coef value is 1.1300000000000001:
Total Profit is 1544.99, total loses is 436
parametr is bull_trend_coef value is 1.1400000000000001:
Total Profit is 1517.46, total loses is 436
parametr is bull_trend_coef value is 1.1500000000000001:
Total Profit is 1493.47, total loses is 434
parametr is bull_trend_coef value is 1.1600000000000001:
Total Profit is 1578.99, total loses is 431
parametr is bull_trend_coef value is 1.1700000000000002:
Total Profit is 1543.42, total loses is 429
parametr is bull_trend_coef value is 1.1800000000000002:
Tota

KeyboardInterrupt: 

In [72]:
stock_name_list = []
stock_name_list += ['GOOG','JPM','XOM','UNH','JNJ','V','AVGO','PG','LLY','MA','HD','CVX','MRK', 
                       'PEP','COST','ABBV','ADBE','KO','CRM','WMT','MCD','CSCO','BAC','PFE','TMO','ACN','NFLX','ABT','AMD','LIN','ORCL','CMCSA',
                       'TXN','DIS','WFC','DHR','PM','NEE','VZ','INTC','RTX','HON','LOW','UPS','INTU','SPGI','NKE','COP','QCOM','BMY','CAT','UNP','BA','ISRG',
                        'GE','IBM','AMGN','AMAT','MDT','SBUX','PLD','NOW','MS','DE','BLK','GS','T','LMT','AXP','BKNG','SYK','ADI','TJX','ELV','MDLZ','GILD','ADP','MMC',
                        'C','AMT','CVS','VRTX','SCHW','LRCX','MO','TMUS','SLB', 'ETN', 'ZTS', 'CI', 'PYPL']

stock_name_list += ['FI','CB','SO','REGN','BSX','EQIX','BDX','PANW','DUK','EOG','MU','AON','ITW','CSX','SNPS','PGR','APD','KLAC','CME','NOC','CDNS','ICE','ATVI',
                       'CL','SHW','WM','HCA','TGT','FCX','FDX','F','ORLY','MMM','CMG','EW','GM','MCK','NXPI','MCO','NSC','HUM','EMR','DXCM','PNC','PH','MPC','APH',
                       'ROP','FTNT','MCHP','PXD','USB','CCI','MAR','MSI','GD','PSA','JCI','PSX','SRE','ADSK','AZO','TDG','ECL','AJG','KMB','TEL','TT','AEP','EL','PCAR',
                       'OXY','TFC','CARR','D','IDXX','GIS','ON','COF','ADM','MNST','NUE','CTAS','AIG','EXC','VLO','MRNA','ANET','WMB','O','STZ','IQV','HLT','CHTR','WELL',
                       'BIIB','SPG','MSCI','DHI','ROK']
len(stock_name_list)

191

In [91]:
for parameter in parameters_list_search:
  print(f'parameter {parameter} best value is {vars()[parameter]}')

  # if df['ha_pct'].iloc[i] > RIV \
  # and reverse_flag==False \
  # and sell_finished == True \
  # and last_top / df['open'].iloc[i] > last_top_ratio \
  # and i - last_top_i > distance_from_last_top \
  # and buy_time_cond \
  # and buy_ratio > buy_ratio_border \
  # and not(is_near_global_max(i, k=400, prt=is_near_global_max_prt)):

parameter bull_trend_coef best value is 1.05


In [108]:
# TRY TO REDUCE NUMBER OF FAIL NOT PROFIT!!!
total_profit_list = []
# parametr_list = np.arange(86, 100, 1)
# parametr_list = [1, 1.001, 1.002, 1.005, 1.007, 1.01, 1.02]
# parametr_list = [30]
# for p in parametr_list:
#   is_near_global_max_prt= p
 
print("TRAIN RESULTS")
total_profit = 0
total_loses = 0
for ticker in stock_name_list:
  file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
  path = os.path.join(parent_path, folder_saved_models, file_name)
  df = load_data_one_stock(path)
  profit, loses = one_stock_simulation(df, ticker, mode = 'train', print_sim_results=True, print_trans_history=True)
  total_profit += profit
  total_loses += loses
print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}')

print("TEST RESULTS")
total_profit = 0
total_loses = 0
for ticker in stock_name_list:
  file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
  path = os.path.join(parent_path, folder_saved_models, file_name)
  df = load_data_one_stock(path)
  profit, loses = one_stock_simulation(df, ticker, mode = 'test', print_sim_results=True, print_trans_history=True)
  total_profit += profit
  total_loses += loses
print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}')


TRAIN RESULTS
--------------------------------------------------
Simulation results for ticker GOOG:
{'number success': 10, 'number fails': 4, 'profit': -39.25965428924562, 'transactions': [{'type': 'fail', 'buy_price': 110.87, 'sell_price': 107.64, 'ratio': 23.77, 'buy_time': Timestamp('2022-06-30 11:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-07-01 09:30:00-0400', tz='America/New_York')}, {'type': 'success', 'buy_price': 111.68, 'sell_price': 114.0, 'ratio': 25.73, 'buy_time': Timestamp('2022-07-14 13:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-07-15 09:30:00-0400', tz='America/New_York')}, {'type': 'fail', 'buy_price': 108.76, 'sell_price': 105.99, 'ratio': 12.24, 'buy_time': Timestamp('2022-07-25 09:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2022-07-26 09:30:00-0400', tz='America/New_York')}, {'type': 'success', 'buy_price': 115.25, 'sell_price': 117.06, 'ratio': 26.43, 'buy_time': Timestamp('2022-08-23 13:30:00-0400', t